In [1]:
#@title Mount google drive

from google.colab import drive
drive.mount('/content/drive')

%cd './drive/MyDrive/hMeNets'

# %cd '../'
# !git clone https://github.com/Selbias/hMeNets.git
# #cd './hMeNets'

Mounted at /content/drive
/content/drive/MyDrive/hMeNets


In [2]:
import os

import numpy as np
import pandas as pd

from collections import Counter
from scipy.io import loadmat
from tqdm.auto import tqdm
from glob import glob

# Data Load

In [3]:
data_path = "../MPIIGaze/Data/Normalized/" # 전처리가 수행된 데이터셋 경로 정의
# mat_path = sorted(glob(os.path.join(data_path,"p0*","day0*.mat")))
mat_path = sorted(glob(os.path.join(data_path,"p**","day**.mat"))) # p1*으로 수집한 데이터 경로 정의
print(f"Find {len(mat_path)} Data") # 전체 mat 파일 수 출력

Find 521 Data


In [4]:
loc_list = ["right", "left"] # 눈 위치 컬럼 정의
dat_list = ["image","gaze","pose"] # 데이터 종류 정의

datas = [] # 데이터를 유형별로 담아둘 임시 리스트 정의
for idx in tqdm(range(len(mat_path))): # mat 파일별로 데이터 추출을 위한 반복문
    cur_mat_path = mat_path[idx].replace("\\","/") # 경로 문자열의 슬라이싱을 위한 텍스트 대치 구문
    par_id = cur_mat_path.split("/")[-2] # 경로 문자열에서 참가자 아이디 정보 슬라이싱 구문
    day_id = cur_mat_path.split("/")[-1].split(".")[0] # 경로 문자열에서 수집 일자 정보 슬라이싱 구문

    mat_data = loadmat(cur_mat_path, squeeze_me = True, struct_as_record = True)['data'] # mat파일을 파이썬에서 읽어들이는 구문
    for loc in loc_list: # 눈의 위치 별로 데이터 추출 하기위한 반복문
        loc_data = mat_data[loc].tolist() # 각 눈 위치의 데이터를 담아두는 리스트 정의

        image_data = loc_data['image'].tolist() # 리스트에서 이미지 정보 추출
        pose_data = loc_data['pose'].tolist() # 리스트에서 머리 위치 정보 추출
        gaze_data = loc_data['gaze'].tolist() # 리스트에서 시선 벡터 정보 추출

        if len(image_data.shape) < 3: # 수집한 일자에서 이미지가 1개일 경우, 차원을 추가하는 구문(1, 36, 60)
            image_data = image_data[np.newaxis,:]

        if len(pose_data.shape) < 2: # 수집한 일자에서 머리 위치가 1개일 경우, 차원을 추가하는 구문(1, 3)
            pose_data = pose_data[np.newaxis,:]

        if len(gaze_data.shape) < 2: # 수집한 일자에서 시선 벡터가 1개일 경우, 차원을 추가하는 구문(1, 3)
            gaze_data = gaze_data[np.newaxis,:]

        for i in range(len(image_data)): # 각 데이터의 종류별로 참가자 id, 수집 일자, 눈 위치, 이미지 정보, 머리 위치 정보, 시선 벡터 정보를 리스트에 추가
            gaze = gaze_data[i].tolist()
            data_list = [par_id, day_id, loc, image_data[i], pose_data[i], gaze[0], gaze[1], gaze[2]]
            datas.append(data_list)

# 리스트에 담아두었던 정보들을 DataFrame으로 생성
data_df = pd.DataFrame(columns=["participant_id","day","eye_location","image","pose","gaze_x","gaze_y","gaze_z"], data=datas)
# data_df.tail(3)

  0%|          | 0/521 [00:00<?, ?it/s]

In [5]:
#@title Image/head pose/gaze mirroring

id_vector = np.asarray(data_df["participant_id"].to_list())
is_right = np.asarray(data_df["eye_location"].to_list()) == "right"
images = np.stack(data_df["image"].to_list())
head_poses = np.stack(data_df["pose"].to_list())
gazes = np.asarray(data_df[["gaze_x", "gaze_y", "gaze_z"]])

for i in tqdm(range(len(id_vector))) :
    if is_right[i] :
        images[i] = np.flip(images[i], 1)
        head_poses[i, 0] *= -1
        gazes[i, 0] *= -1

  0%|          | 0/427316 [00:00<?, ?it/s]

In [7]:
np.save('./data/MPIIGaze_ids', id_vector)
np.save('./data/MPIIGaze_is_right', is_right)
np.save('./data/MPIIGaze_images', images)
np.save('./data/MPIIGaze_head_poses', head_poses)
np.save('./data/MPIIGaze_gazes', gazes)